In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [ ]:
def toUri(path):
    """Return file:// URL from a filename."""
    path = os.path.abspath(os.path.expanduser(os.path.expandvars(path)))
    return pathlib.Path(path).as_uri()

## Es necesario tener HDFS arrancado (start-dfs.sh)

In [ ]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [ ]:
lines = spark \
    .readStream \
    .load( format="socket",
           host="localhost",
           port=9997
         )

words = lines.select(
    explode(split(lines.value, "b")).alias('word')
)


wordCounts = words.groupBy('word').count()

Ejecutar en terminar nc -lk 9999 y escribir algo

In [ ]:
#Conexion a Socket
#muestra el resultado en la consola donde se ejecuto pyspark
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

In [ ]:
#manda el resultado a un topic de kafka
#Creamos topic
#kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic updates --create --partitions 3 --replication-factor 2
#Consumidor
#kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic updates
words2 = lines.select(
        explode(split(lines.value, " ")).alias('value')#Se tiene que llamar value para poder mandarlo a kafka
)

query = words2 \
    .writeStream \
    .format("kafka")  \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "updates") \
    .option("checkpointLocation", "/tmp2") \
    .start()

try:
    query.awaitTermination()
except:
    pass

In [ ]:
#Guarda en tabla en memoria y consulta con SparkSQL
query = wordCounts \
    .writeStream \
    .queryName("aggregates") \
    .outputMode("complete") \
    .format("memory") \
    .start()



try:
    query.awaitTermination()
except:
    pass

In [ ]:
spark.sql("select * from aggregates").show()